In [ ]:
import sys, IPython

install_needed = True
#install_needed = False

if install_needed:
    print("===> Installing deps and restarting kernel. Please change 'install_needed = False' and run this code cell again.")
    !{sys.executable} -m pip install -U sagemaker locust pyngrok torch==1.11 transformers==4.17
    #IPython.Application.instance().kernel.do_shutdown(True)
else:
    import sagemaker, transformers, torch
    print(f'SageMaker API version={sagemaker.__version__}')

In [1]:
!nvidia-smi

Wed Jun  5 00:43:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   25C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import boto3
import sagemaker

# SageMaker 세션과 기본 S3 버킷 초기화
sess = sagemaker.Session()
bucket = sess.default_bucket()

# Boto3를 통한 SageMaker 클라이언트 설정
sm_client = boto3.client("sagemaker")

# 실행 역할 및 리전 설정
role = sagemaker.get_execution_role()
boto_session = boto3.session.Session()
region = boto_session.region_name

# 확인을 위한 출력
print((bucket, role, region))

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
('sagemaker-ap-northeast-2-095298771240', 'arn:aws:iam::095298771240:role/service-role/AmazonSageMaker-ExecutionRole-20240603T133999', 'ap-northeast-2')


In [3]:
!pwd

/home/ec2-user/SageMaker/kordpr/dpr_sagemaker


In [6]:
import os
import tarfile
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# # 모델과 토크나이저 다운로드
# model_name = 'gpt2'
# model = GPT2LMHeadModel.from_pretrained(model_name)
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# # 로컬 디렉터리에 저장
model_dir = "../../kordpr"
# os.makedirs(model_dir, exist_ok=True)
# model.save_pretrained(model_dir)
# tokenizer.save_pretrained(model_dir)

# tar.gz 파일로 압축
with tarfile.open('model.tar.gz', 'w:gz') as tar:
    for file_name in os.listdir(model_dir):
        tar.add(os.path.join(model_dir, file_name), arcname=file_name)

# S3에 업로드
import boto3
s3 = boto3.client('s3')
bucket_name = 'dawoncecds3'
s3.upload_file('model.tar.gz', bucket_name, 'model.tar.gz')


In [4]:
bucket_name = 'dawoncecds3'

In [5]:
from sagemaker.huggingface import HuggingFaceModel

# HuggingFaceModel 객체 생성
huggingface_model = HuggingFaceModel(
    model_data='s3://{}/model.tar.gz'.format(bucket_name),  # S3에 저장된 모델 경로
    role=role,  # SageMaker 역할
    entry_point='inference.py',  # 모델 및 추론 스크립트 파일
    transformers_version='4.17',  # 트랜스포머 라이브러리 버전
    pytorch_version='1.10',  # 파이토치 버전
    py_version='py38',  # 파이썬 버전
    #image_uri='471112711710.dkr.ecr.ap-northeast-2.amazonaws.com/my_pytorch_repository',
)

# 엔드포인트 이름을 고정
endpoint_name = 'dawoncecdEP'

# 엔드포인트 배포
predictor = huggingface_model.deploy(
    initial_instance_count=1, 
    instance_type='ml.g4dn.xlarge',
    endpoint_name=endpoint_name  # 고정된 엔드포인트 이름 지정
)

--------!

In [8]:

# !pip freeze > requirements.txt

In [10]:
predictor.delete_endpoint()

In [13]:
# !pip show transformers torch python

Name: transformers
Version: 4.40.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
---
Name: torch
Version: 2.1.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: captum, torchaudio, tor

In [1]:
!python --version

Python 3.10.14
